In [1]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving my_utils.py to my_utils.py


In [2]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving pre_processing.py to pre_processing.py


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

import copy
import numpy as np
import pandas as pd
import my_utils
from pre_processing import text_clean
from my_utils import drop_constant_columns


def create_cohmetrix_input():
    # reading Fake and Satire data
    data = my_utils.read_fake_satire_dataset("drive/My Drive/Colab Notebooks/data/FakeNewsData/StoryText 2/")

    for index, row in data.iterrows():
        with open('data/cohmetrix/input/d' + str(index) + '_' + str(row[2]) + '.txt', 'w+') as text_file:
            # if we want to c;ean the text
            # text_file.write(text_clean(row[0], True, True, False, 1))
            text_file.write(row[0])
            text_file.close()

    print("Created the input files for Coh-Metrix successfully.")


def create_regresssion_input():
    """
    creating a single excel file as inout to regression analysis using the coh-metrix output
    :return:
    """
    coh_data = pd.read_csv("drive/My Drive/Colab Notebooks/data/cohmetrix/output/satirefake.csv")
    x_columns = list(coh_data.iloc[:, 1:])
    x_reg = []
    y_reg = []

    for item in coh_data.iterrows():
        tmp = item[1][0].split('\\')
        tmp = tmp[len(tmp)-1].split('.')[0].split('_')
        doc_label = int(tmp[1])

        x_reg.append(item[1][1:])
        y_reg.append(doc_label)

    # y_reg = np.array(y_reg)

    # converting list to dataframe
    x_reg = pd.DataFrame(np.array(x_reg).reshape(len(x_reg), len(x_columns)), columns=x_columns)
    # dropping constant value columns
    # x_lin = np.array(x_lin)
    x_reg = drop_constant_columns(x_reg)

    # scaling the data
    # scaler = MinMaxScaler(feature_range=(0, 1))
    # x_lin = scaler.fit_transform(x_lin)

    x_full = copy.deepcopy(x_reg)
    x_full["label"] = y_reg
    x_full = x_full.fillna(0)
    writer = pd.ExcelWriter('drive/My Drive/Colab Notebooks/data/satire_fake_full.xlsx')
    x_full.to_excel(writer, 'Sheet1')
    writer.save()

    print("Created regression input files successfully.")


def logistic_regression(x, y, model_features):
    """
    non-cross validation version of logistic regression
    :param x:
    :param y:
    :param model_features:
    :return:
    """
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
    logreg = LogisticRegression(class_weight="balanced", solver='lbfgs')
    logreg.fit(x_train, y_train)
    print("\n\nLogistic regression report")
    print("=========================")
    if model_features != "":
        # print(logreg.coef_)
        feature_coeff = list(zip(logreg.coef_[0], model_features))
        for item in sorted(feature_coeff, key=lambda x: abs(x[0])):
            print(item)
    y_pred = logreg.predict(x_test)
    print('Accuracy: {:.2f}'.format(logreg.score(x_test, y_test)))
    print(classification_report(y_test, y_pred))
    print("Confusion matrix")
    print(confusion_matrix(y_test, y_pred))


def my_classifier(x, y, clf):
    """
    binary classification using different classifiers and cross validation
    :param x: independent variables
    :param y: dependent variable
    :param clf: classifier
    :return:
    """
    # parts that I changed to retrieve recall/precision/F1 score
    r_scores = cross_val_score(clf, x, y, cv=5, scoring='recall')  #scoring='recall'
    p_scores = cross_val_score(clf, x, y, cv=5, scoring='precision')  #scoring='precision'
    f_scores = cross_val_score(clf, x, y, cv=5, scoring='f1_macro')  #scoring='f1'

    print("Recall: %0.2f (+/- %0.2f)" % (r_scores.mean(), r_scores.std() * 2))
    print("Precision: %0.2f (+/- %0.2f)" % (p_scores.mean(), p_scores.std() * 2))
    print("F1: %0.2f (+/- %0.2f)" % (f_scores.mean(), f_scores.std() * 2))


data = pd.read_csv("drive/My Drive/Colab Notebooks/data/classification.csv")
model_features = list(data)

x = data.iloc[:, 1:len(model_features) - 1]
y = data.iloc[:, len(model_features) - 1]

model_features = list(x)
# naive bayes
print("Naive Bayes")
my_classifier(x, y, GaussianNB())

# svm
print("SVM")
my_classifier(x, y, SVC(kernel='linear', C=1))

# logistic regression
print("Logistic Regression")
my_classifier(x, y, LogisticRegression(class_weight="balanced", solver='lbfgs'))

print("Gradient Boosting")
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(learning_rate=learning_rate)
    my_classifier(x, y, gb)



Naive Bayes
Recall: 0.50 (+/- 0.13)
Precision: 0.73 (+/- 0.19)
F1: 0.67 (+/- 0.08)
SVM
Recall: 0.67 (+/- 0.09)
Precision: 0.72 (+/- 0.06)
F1: 0.73 (+/- 0.06)
Logistic Regression
Recall: 0.73 (+/- 0.10)
Precision: 0.69 (+/- 0.11)
F1: 0.73 (+/- 0.09)
Gradient Boosting
Recall: 0.54 (+/- 0.13)
Precision: 0.60 (+/- 0.14)
F1: 0.63 (+/- 0.06)
Recall: 0.56 (+/- 0.07)
Precision: 0.61 (+/- 0.07)
F1: 0.63 (+/- 0.04)
Recall: 0.53 (+/- 0.14)
Precision: 0.61 (+/- 0.04)
F1: 0.62 (+/- 0.05)
Recall: 0.60 (+/- 0.11)
Precision: 0.59 (+/- 0.05)
F1: 0.63 (+/- 0.05)
Recall: 0.53 (+/- 0.07)
Precision: 0.62 (+/- 0.09)
F1: 0.62 (+/- 0.08)
Recall: 0.55 (+/- 0.09)
Precision: 0.57 (+/- 0.07)
F1: 0.61 (+/- 0.06)
